# RMSNorm Operation

This notebook shows how to compute an RMS norm using the cuDNN python frontend.

$$\text{RMSNorm}(x) = \frac{x}{\sqrt{\mathbb{E}(x^2) + \epsilon}}\cdot\gamma+\beta$$

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NVIDIA/cudnn-frontend/blob/main/samples/python/28_rmsnorm.ipynb)

## Prerequisites and Setup
This notebook requires an NVIDIA GPU. If `nvidia-smi` fails, go to Runtime -> Change runtime type -> Hardware accelerator and confirm a GPU is selected.

In [ ]:
# get_ipython().system('nvidia-smi')

If running on Colab, you will need to install the cudnn python interface.

In [ ]:
# get_ipython().system('pip install nvidia-cudnn-cu12')
# get_ipython().system('pip install nvidia-cudnn-frontend')
# get_ipython().system('pip3 install --pre torch --index-url https://download.pytorch.org/whl/nightly/cu128')

## Overview

In the following, we are going to perform the RMS norm forward pass with the following problem sizes:

- batch size: 4
- sequence length: 1024
- hidden dimension: 768

The tensor will be in 16-bit float format.

In [ ]:
import cudnn
import torch

torch.manual_seed(1)

handle = cudnn.create_handle()
print("Running with cudnn backend version:", cudnn.backend_version())

assert torch.cuda.is_available()
assert (
    cudnn.backend_version_string() >= "9.1.0"
), "RMSNorm requires cuDNN 9.1.0 or higher"

batch, seq_length, hidden_size = 4, 1024, 128
dtype = torch.float16
# Epsilon is a small number to prevent division by 0.
epsilon_value = 1e-3

## Using Wrapper

Below is how you can use the `Graph` wrapper to perform RMS norm with the input tensors in PyTorch format.

In [ ]:
# forward pass of rmsnorm using PyTorch
x_gpu = torch.randn(
    batch * seq_length,
    hidden_size,
    1,
    1,
    device="cuda",
    dtype=dtype,
    requires_grad=True,
).to(memory_format=torch.channels_last)
scale_gpu = torch.randn(
    1, hidden_size, 1, 1, device="cuda", dtype=dtype, requires_grad=True
).to(memory_format=torch.channels_last)
bias_gpu = torch.randn(
    1, hidden_size, 1, 1, device="cuda", dtype=dtype, requires_grad=True
).to(memory_format=torch.channels_last)
eps_cpu = torch.full((1, 1, 1, 1), epsilon_value, dtype=torch.float32, device="cpu")

# forward pass of rmsnorm using cuDNN graph
with cudnn.Graph(
    intermediate_data_type=cudnn.data_type.FLOAT,
    compute_data_type=cudnn.data_type.FLOAT,
    inputs=["rms_fwd::input", "rms_fwd::scale", "rms_fwd::bias", "rms_fwd::epsilon"],
    outputs=["rms_fwd::Y", "rms_fwd::INV_VARIANCE"],
) as rmsnorm_graph:
    out, inv_var = rmsnorm_graph.rmsnorm(
        name="rms_fwd",
        norm_forward_phase=cudnn.norm_forward_phase.TRAINING,
        input=x_gpu,
        scale=scale_gpu,
        bias=bias_gpu,
        epsilon=eps_cpu,
    )
    # set output, inv_var must be float32 tensor
    out.set_output(True).set_data_type(dtype)
    inv_var.set_output(True).set_data_type(cudnn.data_type.FLOAT)

# execute the graph and retrieve the output tensors
out_gpu, inv_var_gpu = rmsnorm_graph(x_gpu, scale_gpu, bias_gpu, eps_cpu, handle=handle)

# verify the result with PyTorch API
out_ref = torch.nn.functional.rms_norm(
    x_gpu,
    [hidden_size, 1, 1],  # RMS norm over last 3 dimensions
    weight=scale_gpu.squeeze(0),
    eps=epsilon_value,
)
out_ref = out_ref + bias_gpu
inv_var_ref = torch.rsqrt(
    torch.mean(x_gpu.float().pow(2), dim=(1, 2, 3), keepdim=True) + epsilon_value
)

torch.testing.assert_close(out_gpu, out_ref, atol=5e-3, rtol=3e-3)
torch.testing.assert_close(inv_var_gpu, inv_var_ref, atol=5e-3, rtol=3e-3)

The above performs forward pass of RMSNorm. Next, we will perform the backward pass:

In [ ]:
# Compute gradients: Ask PyTorch not to discard the grads after use so that we can read it twice
# out_ref.grad will be used in the cudnn graph, x_gpu.grad, scale_gpu.grad, and bias_gpu.grad will
# be used to compare with the cudnn graph output.
target = torch.randn_like(out_ref)
criterion = torch.nn.MSELoss()
loss = criterion(out_ref, target)

out_ref.retain_grad()
x_gpu.retain_grad()
scale_gpu.retain_grad()
bias_gpu.retain_grad()

loss.backward()

# Backward pass
with cudnn.Graph(
    io_data_type=cudnn.data_type.HALF,
    intermediate_data_type=cudnn.data_type.FLOAT,
    compute_data_type=cudnn.data_type.FLOAT,
    inputs=[
        "rms_bwd::grad",
        "rms_bwd::input",
        "rms_bwd::scale",
        "rms_bwd::inv_variance",
    ],
    outputs=["rms_bwd::DX", "rms_bwd::Dscale", "rms_bwd::Dbias"],
) as bwd_graph:
    dx, dscale, dbias = bwd_graph.rmsnorm_backward(
        name="rms_bwd",
        grad=out_ref,
        input=x_gpu,
        scale=scale_gpu,
        inv_variance=inv_var_gpu,
        has_dbias=True,
    )
    dx.set_output(True).set_data_type(x_gpu.dtype)
    dscale.set_output(True).set_data_type(x_gpu.dtype)
    dbias.set_output(True).set_data_type(x_gpu.dtype)

# execute the graph and retrieve the output tensors
dx_gpu, dscale_gpu, dbias_gpu = bwd_graph(
    out_ref.grad, x_gpu, scale_gpu, inv_var_gpu, handle=handle
)

torch.testing.assert_close(x_gpu.grad, dx_gpu, atol=5e-3, rtol=3e-3)
torch.testing.assert_close(scale_gpu.grad, dscale_gpu, atol=5e-3, rtol=3e-3)
torch.testing.assert_close(bias_gpu.grad, dbias_gpu, atol=5e-3, rtol=3e-3)

## Using Python Binding APIs

#### RMS norm forward pass

Create input tensor GPU buffers. We use PyTorch to allocate GPU tensors so we can reuse them easily when we calculate reference outputs.

In [ ]:
# input tensor memory, initialize them to random numbers
x_gpu = torch.randn(
    batch * seq_length,
    hidden_size,
    1,
    1,
    device="cuda",
    dtype=dtype,
    requires_grad=True,
).to(memory_format=torch.channels_last)
scale_gpu = torch.randn(
    1, hidden_size, 1, 1, device="cuda", dtype=dtype, requires_grad=True
).to(memory_format=torch.channels_last)
bias_gpu = torch.randn(
    1, hidden_size, 1, 1, device="cuda", dtype=dtype, requires_grad=True
).to(memory_format=torch.channels_last)
# set epsilon to epsilon_value, allocate on cpu.
eps_cpu = torch.full((1, 1, 1, 1), epsilon_value, dtype=torch.float32, device="cpu")

Create cuDNN graph and cuDNN tensors

In [ ]:
graph = cudnn.pygraph(
    handle=handle,
    intermediate_data_type=cudnn.data_type.FLOAT,
    compute_data_type=cudnn.data_type.FLOAT,
)

# create tensor handles with the graph API
x = graph.tensor_like(x_gpu.detach()).set_name("X")
scale = graph.tensor_like(scale_gpu.detach()).set_name("scale")
bias = graph.tensor_like(bias_gpu.detach()).set_name("bias")
epsilon = graph.tensor_like(eps_cpu).set_name("epsilon")

out, inv_var = graph.rmsnorm(
    name="rmsnorm",
    input=x,
    norm_forward_phase=cudnn.norm_forward_phase.TRAINING,
    scale=scale,
    bias=bias,
    epsilon=epsilon,
)

# enable all outputs
out.set_name("output").set_output(True).set_data_type(dtype)
inv_var.set_name("inv_var").set_output(True).set_data_type(cudnn.data_type.FLOAT);

Build and execute the graph

In [ ]:
# Build the graph
graph.build([cudnn.heur_mode.A, cudnn.heur_mode.FALLBACK])

# Mapping of (handles -> memory)
variant_pack = {
    x: x_gpu.detach(),
    scale: scale_gpu.detach(),
    bias: bias_gpu.detach(),
    epsilon: eps_cpu,
    out: out_gpu,
    inv_var: inv_var_gpu,
}

# Execute the graph
workspace = torch.empty(graph.get_workspace_size(), device="cuda", dtype=torch.uint8)
graph.execute(variant_pack, workspace)
torch.cuda.synchronize()

Compute reference ouputs and verify

In [ ]:
# reference output using PyTorch
out_ref = torch.nn.functional.rms_norm(
    x_gpu,
    [hidden_size, 1, 1],  # RMS norm over last 3 dimensions
    weight=scale_gpu.squeeze(0),
    eps=epsilon_value,
)
out_ref = out_ref + bias_gpu
inv_var_ref = torch.rsqrt(
    torch.mean(x_gpu.float().pow(2), dim=(1, 2, 3), keepdim=True) + epsilon_value
)

# Test cuDNN's output against PyTorch's and check correctness
torch.testing.assert_close(out_gpu, out_ref, rtol=5e-3, atol=5e-3)
torch.testing.assert_close(inv_var_gpu, inv_var_ref, rtol=5e-3, atol=5e-3)

#### RMSNorm Backwards Pass

In [ ]:
target = torch.randn_like(out_ref)
criterion = torch.nn.MSELoss()
loss = criterion(out_ref, target)

out_ref.retain_grad()
x_gpu.retain_grad()
scale_gpu.retain_grad()
bias_gpu.retain_grad()

loss.backward()

In [ ]:
bwd_graph = cudnn.pygraph(
    handle=handle,
    intermediate_data_type=cudnn.data_type.FLOAT,
    compute_data_type=cudnn.data_type.FLOAT,
)

d_out = bwd_graph.tensor_like(out_ref.grad)
x_bwd = bwd_graph.tensor_like(x, name="x")
scale_bwd = bwd_graph.tensor_like(scale, name="scale")
inv_var_bwd = bwd_graph.tensor_like(inv_var, name="inv_var")

d_x, d_scale, d_bias = bwd_graph.rmsnorm_backward(
    name="d_rmsnorm",
    grad=d_out,
    input=x_bwd,
    scale=scale_bwd,
    inv_variance=inv_var_bwd,
    has_dbias=True,
)

d_x.set_output(True).set_data_type(dtype)
d_scale.set_output(True).set_data_type(dtype)
d_bias.set_output(True).set_data_type(dtype);

In [ ]:
# Build the bwd_graph
bwd_graph.build([cudnn.heur_mode.A, cudnn.heur_mode.FALLBACK])

# Create output buffers for gradients
d_x_gpu = torch.empty_like(x_gpu)
d_scale_gpu = torch.empty_like(scale_gpu)
d_bias_gpu = torch.empty_like(bias_gpu)

# Execute the graph
workspace = torch.empty(
    bwd_graph.get_workspace_size(), device="cuda", dtype=torch.uint8
)
bwd_graph.execute(
    {
        x_bwd: x_gpu.detach(),
        scale_bwd: scale_gpu.detach(),
        d_out: out_ref.grad,
        inv_var_bwd: inv_var_gpu.detach(),
        d_x: d_x_gpu,
        d_scale: d_scale_gpu,
        d_bias: d_bias_gpu,
    },
    workspace,
    handle=handle,
)

Compare results and check correctness

In [ ]:
torch.cuda.synchronize()

torch.testing.assert_close(x_gpu.grad, d_x_gpu, atol=2e-4, rtol=2e-4)
torch.testing.assert_close(scale_gpu.grad, d_scale_gpu, atol=2e-4, rtol=2e-4)
torch.testing.assert_close(bias_gpu.grad, d_bias_gpu, atol=2e-4, rtol=2e-4)

Perform Cleanup

In [ ]:
cudnn.destroy_handle(handle)